1. Import


In [13]:
import pandas as pd
import numpy as np
import pickle
import yaml
from pathlib import Path
from typing import List, Dict, Any
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

2. Analysis Settings


In [ ]:
CONFIG_PATH = "../configs/movielens_1m.yaml"

config_path = Path(CONFIG_PATH)
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

DATASET_TO_ANALYZE = "movielens_1m" 
SAVE_DIR = Path("../results") 
MODEL_NAMES = ["WMF", "EASEᴿ", "VAECF", "NeuMF"]
DATASET_NAME = config['dataset_name']
DIVERSIFICATION_MODES = ["None", "I_DIV", "A_DIV", "I_DIV+A_DIV"]
PERSONALIZE_MODES = ["P", "NP"]
METRIC_NAMES = ["NDCG@10", "Recall@10", "Precision@10", "MRR@10", "Novelty", "CatalogCoverage", "APLT", "ACLT"]
BASELINE_MODE = "None"
TODAY_DATE = pd.Timestamp.now().strftime("%Y%m%d")

print(f"데이터셋 분석 시작: {DATASET_NAME}")

데이터셋 분석 시작: movielens_1m


3. Helper Functions


In [15]:
def load_results_to_dataframe(
    dataset: str,
    model_names: List[str],
    diversification_modes: List[str],
    personalize_modes: List[str],
    save_dir: Path
) -> pd.DataFrame:
    
    records = []
    dataset_path = save_dir / dataset
    print(f"결과 로드 시작: 경로 {dataset_path.resolve()}")

    BASELINE_MODE = "None" 

    for model in model_names:
        for div_mode in diversification_modes:
            for per_mode in personalize_modes: 
                
                avg_results_data = None
                
                if div_mode == BASELINE_MODE:
                    file_key = "None"
                else:
                    file_key = f"{div_mode}_{per_mode}"

                file_path = dataset_path / f"{model}_result_{file_key}_avg.pkl" 
                
                try:
                    with open(file_path, "rb") as f:
                        avg_results_data = pickle.load(f)
                    
                    if avg_results_data:
                        for metric, score in avg_results_data.items():
                            records.append({
                                "model": model,
                                "diversification": div_mode,
                                "personalize": per_mode, 
                                "metric": metric,
                                "score": score
                            })

                except FileNotFoundError:
                    print(f"정보: 평균 파일 찾을 수 없음: {file_path.name}")
                except Exception as e:
                    print(f"오류: {file_path.name} 로드 실패: {e}")

    if not records:
        print("\n--- 에러: 결과 없음 ---")
        
    print(f"총 {len(records)}개의 평균 결과 레코드 로드 완료.")
    return pd.DataFrame(records)

def summarize_and_format_results(
    df: pd.DataFrame, 
    model_names: List[str],
    diversification_modes: List[str],
    PERSONALIZE_MODES: List[str], 
    metric_names: List[str],
    baseline_mode: str
) -> pd.DataFrame:
    
    print("평균(AVG) 결과 요약 및 포맷팅 시작")

    df['model'] = pd.Categorical(df['model'], categories=model_names, ordered=True)
    df['diversification'] = pd.Categorical(df['diversification'], categories=diversification_modes, ordered=True)
    df['personalize'] = pd.Categorical(df['personalize'], categories=PERSONALIZE_MODES, ordered=True) 
    df['metric'] = pd.Categorical(df['metric'], categories=metric_names, ordered=True)

    try:
        summary_pivot = df.pivot_table(
            index=["model", "diversification", "personalize"],
            columns="metric",
            values="score"
        )
    except Exception as e:
        print(f"오류: 피벗 테이블 생성 실패. {e}")
        return pd.DataFrame()
    
    rel_changes = pd.DataFrame(index=summary_pivot.index, columns=summary_pivot.columns)
    
    for model in summary_pivot.index.get_level_values('model').unique():
        for p_mode in summary_pivot.index.get_level_values('personalize').unique(): 
            
            base_index = (model, baseline_mode, p_mode)
            
            if base_index in summary_pivot.index:
                base_means = summary_pivot.loc[base_index] 
                
                comparison_idx = (summary_pivot.index.get_level_values('model') == model) & \
                                 (summary_pivot.index.get_level_values('personalize') == p_mode) & \
                                 (summary_pivot.index.get_level_values('diversification') != baseline_mode)
                
                if comparison_idx.any():
                    current_means = summary_pivot.loc[comparison_idx]
                    
                    pct_change = np.where(
                        base_means.values == 0, np.nan, 
                        ((current_means - base_means.values) / base_means.values) * 100
                    )
                    pct_change_df = pd.DataFrame(pct_change, index=current_means.index, columns=current_means.columns)
                    formatted_changes = pct_change_df.applymap(
                        lambda x: f" ({x:+.1f}%)" if pd.notna(x) else " (N/A)"
                    )
                    rel_changes.loc[comparison_idx] = formatted_changes

    final_table = pd.DataFrame(index=summary_pivot.index)
    metric_cols = [col for col in metric_names if col in summary_pivot.columns]

    for metric in metric_cols:
        mean_str = summary_pivot[metric].map('{:.4f}'.format)
        rel_change_str = rel_changes[metric].fillna('')
        final_table[metric] = mean_str + rel_change_str

    rows_to_drop = []
    p_mode_to_drop = None
    if len(PERSONALIZE_MODES) > 1:
        p_mode_to_drop = PERSONALIZE_MODES[1] 
        for model in final_table.index.get_level_values('model').unique():
            idx_to_drop = (model, baseline_mode, p_mode_to_drop)
            if idx_to_drop in final_table.index:
                rows_to_drop.append(idx_to_drop)

    final_table_cleaned = final_table.drop(index=rows_to_drop)

    new_idx_tuples = []
    p_mode_to_rename = PERSONALIZE_MODES[0] 
    for (model, div, pers) in final_table_cleaned.index:
        if div == baseline_mode and pers == p_mode_to_rename:
            new_idx_tuples.append((model, div, '-'))
        else:
            new_idx_tuples.append((model, div, pers))
            
    final_table_cleaned.index = pd.MultiIndex.from_tuples(
        new_idx_tuples, 
        names=final_table_cleaned.index.names
    )
    
    print("결과 요약 및 포맷팅 완료.")
    return final_table_cleaned

4. Analysis


In [16]:
all_results_df = load_results_to_dataframe(
    dataset=DATASET_TO_ANALYZE,
    model_names=MODEL_NAMES, 
    diversification_modes=DIVERSIFICATION_MODES, 
    personalize_modes=PERSONALIZE_MODES,  
    save_dir=SAVE_DIR
)

if not all_results_df.empty:
    final_summary_df = summarize_and_format_results(
        all_results_df,
        model_names=MODEL_NAMES,
        diversification_modes=DIVERSIFICATION_MODES,
        PERSONALIZE_MODES=PERSONALIZE_MODES, 
        metric_names=METRIC_NAMES,
        baseline_mode=BASELINE_MODE
    )
    
    display(final_summary_df)
    
    dataset_save_path = SAVE_DIR / DATASET_TO_ANALYZE
    dataset_save_path.mkdir(parents=True, exist_ok=True)
    output_path = dataset_save_path / f"{TODAY_DATE}_{DATASET_TO_ANALYZE}_experiment_summary.csv" 
    final_summary_df.to_csv(output_path, encoding="utf-8-sig")
    print(f"\n테이블 CSV 파일로 저장됨:\n{output_path.resolve()}")
    
else:
    print("데이터를 로드하지 못해 요약 테이블을 생성 불가")

결과 로드 시작: 경로 /Users/leeheejun/Desktop/대학원 관련/테크니컬 포트폴리오/Technical_Portfolio/results/movielens_1m
총 256개의 평균 결과 레코드 로드 완료.
평균(AVG) 결과 요약 및 포맷팅 시작
결과 요약 및 포맷팅 완료.


NDCG@10        Recall@10     Precision@10           MRR@10           Novelty  CatalogCoverage              APLT              ACLT
model diversification personalize                                                                                                                                           
WMF   None            -                     0.1205           0.0541           0.1108           0.2437            9.6715           0.4669            0.1158            0.2994
      I_DIV           P             0.1165 (-3.3%)   0.0532 (-1.6%)   0.1096 (-1.1%)   0.2360 (-3.2%)    9.6154 (-0.6%)  0.6131 (+31.3%)    0.1059 (-8.6%)    0.2898 (-3.2%)
                      NP            0.1176 (-2.4%)   0.0536 (-0.9%)   0.1105 (-0.3%)   0.2378 (-2.4%)    9.6029 (-0.7%)  0.6168 (+32.1%)   0.1033 (-10.8%)    0.2871 (-4.1%)
      A_DIV           P            0.0876 (-27.3%)  0.0452 (-16.5%)  0.0820 (-26.0%)  0.1864 (-23.5%)  10.7706 (+11.4%)  0.3989 (-14.6%)  0.3975 (+243.3%)  0.6065 (+102.6%)
                      NP           0.0872 (-27.6%)  0.0439 (-18.8%)  0.0813 (-26.6%)  0.1855 (-23.9%)  10.7924 (+11.6%)  0.3916 (-16.1%)  0.3891 (+236.1%)   0.4968 (+65.9%)
      I_DIV+A_DIV     P            0.1031 (-14.4%)   0.0503 (-7.0%)  0.0945 (-14.7%)   0.2207 (-9.5%)   10.2105 (+5.6%)  0.5227 (+12.0%)  0.2539 (+119.3%)   0.4333 (+44.8%)
                      NP           0.1057 (-12.3%)   0.0501 (-7.3%)  0.0964 (-13.0%)   0.2267 (-7.0%)   10.1601 (+5.1%)  0.5252 (+12.5%)   0.2286 (+97.5%)   0.3853 (+28.7%)
EASEᴿ None            -                     0.1412           0.0663           0.1278           0.2798            8.9867           0.5520            0.0066            0.0539
      I_DIV           P             0.1403 (-0.6%)   0.0656 (-1.0%)   0.1277 (-0.1%)   0.2795 (-0.1%)    8.9839 (-0.0%)  0.6161 (+11.6%)    0.0060 (-8.3%)    0.0515 (-4.4%)
                      NP            0.1405 (-0.5%)   0.0660 (-0.5%)   0.1280 (+0.2%)   0.2785 (-0.5%)    8.9799 (-0.1%)  0.6157 (+11.5%)    0.0060 (-8.8%)    0.0505 (-6.3%)
      A_DIV           P             0.1408 (-0.2%)   0.0681 (+2.7%)   0.1275 (-0.2%)   0.2786 (-0.4%)    9.2444 (+2.9%)   0.5278 (-4.4%)  0.0382 (+479.4%)  0.2107 (+291.1%)
                      NP            0.1409 (-0.2%)   0.0678 (+2.3%)   0.1277 (-0.0%)   0.2783 (-0.5%)    9.1864 (+2.2%)   0.5315 (-3.7%)  0.0236 (+258.5%)  0.1299 (+141.1%)
      I_DIV+A_DIV     P             0.1420 (+0.6%)   0.0672 (+1.3%)   0.1285 (+0.6%)   0.2826 (+1.0%)    9.0832 (+1.1%)   0.5808 (+5.2%)  0.0154 (+133.9%)   0.1043 (+93.7%)
                      NP            0.1415 (+0.2%)   0.0667 (+0.6%)   0.1281 (+0.2%)   0.2819 (+0.8%)    9.0644 (+0.9%)   0.5824 (+5.5%)   0.0117 (+77.1%)   0.0801 (+48.7%)
VAECF None            -                     0.1300           0.0547           0.1195           0.2504            9.3317           0.4858            0.0345            0.1599
      I_DIV           P             0.1294 (-0.4%)   0.0548 (+0.3%)   0.1194 (-0.1%)   0.2535 (+1.2%)    9.3103 (-0.2%)  0.5455 (+12.3%)    0.0328 (-4.7%)    0.1565 (-2.1%)
                      NP            0.1293 (-0.5%)   0.0550 (+0.5%)   0.1195 (+0.0%)   0.2517 (+0.5%)    9.3078 (-0.3%)  0.5444 (+12.1%)    0.0326 (-5.4%)    0.1544 (-3.4%)
      A_DIV           P             0.1254 (-3.5%)   0.0535 (-2.2%)   0.1143 (-4.3%)   0.2452 (-2.1%)    9.5492 (+2.3%)   0.4650 (-4.3%)  0.0772 (+124.1%)   0.2584 (+61.6%)
                      NP            0.1263 (-2.8%)   0.0537 (-1.8%)   0.1156 (-3.3%)   0.2450 (-2.1%)    9.4890 (+1.7%)   0.4681 (-3.6%)   0.0598 (+73.3%)   0.2185 (+36.7%)
      I_DIV+A_DIV     P             0.1281 (-1.4%)   0.0544 (-0.5%)   0.1173 (-1.9%)   0.2519 (+0.6%)    9.4147 (+0.9%)   0.5094 (+4.9%)   0.0509 (+47.7%)   0.2073 (+29.6%)
                      NP            0.1282 (-1.3%)   0.0544 (-0.4%)   0.1178 (-1.5%)   0.2503 (-0.0%)    9.3920 (+0.6%)   0.5085 (+4.7%)   0.0439 (+27.5%)   0.1848 (+15.6%)
NeuMF None            -                     0.1361           0.0


테이블 CSV 파일로 저장됨:
/Users/leeheejun/Desktop/대학원 관련/테크니컬 포트폴리오/Technical_Portfolio/results/movielens_1m/20251112_movielens_1m_experiment_summary.csv
